In [47]:
from census import Census
from us import states
import pandas as pd
import geopandas as gpd
import duckdb as duck

In [48]:
api_key = '19744ba94a6c4a1e1c832a117d3a932a6043451b'

In [49]:
c = Census(api_key)

In [118]:
variables = [
'NAME',
'GEO_ID',
'B01001H_001E',
'B01001H_002E',
'B01001H_017E',
'B01001I_001E',
'B01001I_002E',
'B01001I_017E',
'B01002_002E',
'B01002_003E',
'B01003_001E',
'B19013_001E',
'B15003_021E',
'B15003_022E',
'B15003_023E',
'B15003_025E',
'B01001_002E',
'B01001_026E',
'B01001B_001E',
'B01001B_002E',
'B01001B_017E'
]

columns = [
'COUNTY',
'FIPS',
'POPULATION_WHITE_NOT_LATINO_ALL',
'POPULATION_WHITE_NOT_LATINO_MALE',
'POPULATION_WHITE_NOT_LATINO_FEMALE',
'POPULATION_HISPANIC_OR_LATINO_ALL',
'POPULATION_HISPANIC_OR_LATINO_MALE',
'POPULATION_HISPANIC_OR_LATINO_FEMALE',
'AGE_MEDIAN_ALL_MALE',
'AGE_MEDIAN_ALL_FEMALE',
'POPULATION_TOTAL',
'INCOME_MEDIAN_INCOME_12_MONTH',
'EDUCATION_ASSOCIATE_DEGREE',
'EDUCATION_BACHELOR_DEGREE',
'EDUCATION_MASTER_DEGREE',
'EDUCATION_DOCTORATE_DEGREE',
'POPULATION_TOTAL_MALE',
'POPULATION_TOTAL_FEMALE',
'POPULATION_BLACK_ALL',
'POPULATION_BLACK_MALE',
'POPULATION_BLACK_FEMALE',
'GEO_STATE_FIPS',
'GEO_COUNTY_FIPS'
]
df = pd.DataFrame(columns=columns)

In [119]:
year = [2012, 2016, 2020, 2022]

for y in year:
    data = c.acs5.get(variables, {'for': 'county:*'}, year=y)
    df_temp = pd.DataFrame(data)
    df_temp.columns = columns
    df_temp['YEAR'] = y
    df = pd.concat([df, df_temp])

C:\Users\appar\AppData\Local\Temp\ipykernel_45792\1917482527.py:8: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_temp])


In [120]:
census_county = df.copy()

In [121]:
census_county.shape

(12884, 24)

In [122]:
census_county = census_county.loc[census_county.GEO_STATE_FIPS != '72']

In [123]:
census_county['POPULATION_DEGREE'] = census_county.EDUCATION_ASSOCIATE_DEGREE + census_county.EDUCATION_BACHELOR_DEGREE + census_county.EDUCATION_MASTER_DEGREE + census_county.EDUCATION_DOCTORATE_DEGREE

In [128]:
census_county.head()

,COUNTY,FIPS,POPULATION_WHITE_NOT_LATINO_ALL,POPULATION_WHITE_NOT_LATINO_MALE,POPULATION_WHITE_NOT_LATINO_FEMALE,POPULATION_HISPANIC_OR_LATINO_ALL,POPULATION_HISPANIC_OR_LATINO_MALE,POPULATION_HISPANIC_OR_LATINO_FEMALE,AGE_MEDIAN_ALL_MALE,AGE_MEDIAN_ALL_FEMALE,...,INCOME_MEDIAN_INCOME_12_MONTH,POPULATION_TOTAL_MALE,POPULATION_TOTAL_FEMALE,POPULATION_BLACK_ALL,POPULATION_BLACK_MALE,POPULATION_BLACK_FEMALE,GEO_STATE_FIPS,GEO_COUNTY_FIPS,YEAR,POPULATION_DEGREE
0,"Linn County, Missouri",0500000US29115,12201.0,5862.0,6339.0,196.0,84.0,112.0,42.0,43.7,...,39028.0,6059.0,6609.0,75.0,34.0,41.0,29,115,2012.0,1405.0
1,"Howell County, Missouri",0500000US29091,38396.0,18592.0,19804.0,698.0,313.0,385.0,38.1,41.8,...,34148.0,19477.0,20853.0,136.0,88.0,48.0,29,091,2012.0,5460.0
2,"Johnson County, Missouri",0500000US29101,46589.0,23553.0,23036.0,1687.0,857.0,830.0,28.9,30.8,...,47960.0,26826.0,26138.0,2374.0,1230.0,1144.0,29,101,2012.0,9801.0
3,"Laclede County, Missouri",0500000US29105,33571.0,16490.0,17081.0,707.0,397.0,310.0,37.9,40.7,...,39101.0,17526.0,17981.0,234.0,186.0,48.0,29,105,2012.0,4136.0
4,"Maries County, Missouri",0500000US29125,8833.0,4438.0,4395.0,83.0,30.0,53.0,41.7,43.6,...,44885.0,4568.0,4572.0,76.0,26.0,50.0,29,125,2012.0,1431.0


In [129]:
# Load the shapefile
counties_raw = gpd.read_file("C:/Users/appar/OneDrive/jup_nb/cb_2018_us_county_500k/cb_2018_us_county_500k.shp",)
counties_raw = counties_raw.to_crs(epsg=2163)
counties = counties_raw.copy()
# Calculate area of each county and sort
counties['area_sqm'] = counties.ALAND + counties.AWATER
counties['area_sqmi'] = counties['area_sqm'] / 2.59e+6  # convert to square miles
counties['area_sqkm'] = counties['area_sqm'] / 10**6

In [130]:
county_size = pd.DataFrame(counties)

In [131]:
county_size.loc[county_size.NAME == 'King']

,STATEFP,COUNTYFP,COUNTYNS,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,geometry,area_sqm,area_sqmi,area_sqkm
1036,53,033,01531933,0500000US53033,53033,King,06,5478600963,497675093,"MULTIPOLYGON (((-1653112.417 530493.679, -1652...",5976276056,2307.442493,5976.276056
2476,48,269,01383920,0500000US48269,48269,King,06,2359148814,6398044,"POLYGON ((-48135.615 -1239308.947, -6617.449 -...",2365546858,913.338555,2365.546858


In [132]:
county_size = county_size.loc[:,['STATEFP', 'COUNTYFP', 'AFFGEOID', 'NAME','ALAND','AWATER','area_sqm', 'area_sqmi','area_sqkm']]

In [133]:
county_size['FIPS'] = county_size['AFFGEOID']

In [134]:
county_size.drop('AFFGEOID', inplace=True, axis=1)

In [135]:
census_county.describe()

,POPULATION_WHITE_NOT_LATINO_ALL,POPULATION_WHITE_NOT_LATINO_MALE,POPULATION_WHITE_NOT_LATINO_FEMALE,POPULATION_HISPANIC_OR_LATINO_ALL,POPULATION_HISPANIC_OR_LATINO_MALE,POPULATION_HISPANIC_OR_LATINO_FEMALE,AGE_MEDIAN_ALL_MALE,AGE_MEDIAN_ALL_FEMALE,POPULATION_TOTAL,INCOME_MEDIAN_INCOME_12_MONTH,POPULATION_TOTAL_MALE,POPULATION_TOTAL_FEMALE,POPULATION_BLACK_ALL,POPULATION_BLACK_MALE,POPULATION_BLACK_FEMALE,YEAR,POPULATION_DEGREE
count,1.257200e+04,1.257200e+04,1.257200e+04,1.257200e+04,1.257200e+04,1.257200e+04,12572.000000,12572.000000,1.257200e+04,1.257200e+04,1.257200e+04,1.257200e+04,1.257200e+04,12572.000000,12572.000000,12572.000000,1.257200e+04
mean,6.247252e+04,3.083713e+04,3.163538e+04,1.804496e+04,9.136802e+03,8.908160e+03,39.914548,42.426185,1.022402e+05,-5.307486e+04,5.041385e+04,5.182635e+04,1.285266e+04,6150.754852,6701.903595,2017.500477,2.609830e+04
std,1.416309e+05,6.995733e+04,7.170042e+04,1.232873e+05,6.164675e+04,6.165977e+04,5.402545,5.491002,3.262654e+05,8.408907e+06,1.605400e+05,1.657667e+05,5.513403e+04,25514.927831,29640.905930,3.840912,8.942344e+04
min,2.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,17.900000,20.400000,5.000000e+01,-6.666667e+08,3.400000e+01,1.600000e+01,0.000000e+00,0.000000,0.000000,2012.000000,3.000000e+00
25%,8.041000e+03,4.060000e+03,3.990750e+03,3.287500e+02,1.730000e+02,1.420000e+02,36.700000,39.300000,1.094175e+04,4.223625e+04,5.508000e+03,5.432000e+03,9.800000e+01,60.000000,30.000000,2015.000000,1.777000e+03
50%,2.003300e+04,1.000000e+04,1.004050e+04,1.056500e+03,5.790000e+02,4.850000e+02,39.700000,42.500000,2.576550e+04,5.057100e+04,1.285150e+04,1.289000e+04,7.800000e+02,476.000000,268.000000,2020.000000,4.527500e+03
75%,5.348925e+04,2.640650e+04,2.710100e+04,4.888500e+03,2.599500e+03,2.290250e+03,42.800000,45.700000,6.760625e+04,6.055750e+04,3.375125e+04,3.417650e+04,5.426750e+03,2818.500000,2643.500000,2022.000000,1.405550e+04
max,2.731605e+06,1.369932e+06,1.361673e+06,4.861648e+06,2.428314e+06,2.437468e+06,68.800000,68.900000,1.005716e+07,1.704630e+05,4.954616e+06,5.102539e+06,1.284262e+06,583368.000000,700894.000000,2022.000000,2.694566e+06


In [136]:
census_data_size = pd.merge(census_county, county_size.loc[:,['FIPS','ALAND','AWATER','area_sqm','area_sqmi','area_sqkm']], how='left')

In [138]:
county_size.loc[county_size.STATEFP == '09']

,STATEFP,COUNTYFP,NAME,ALAND,AWATER,area_sqm,area_sqmi,area_sqkm,FIPS
56,09,009,New Haven,1565688367,667081242,2232769609,862.073208,2232.769609,0500000US09009
659,09,001,Fairfield,1618651428,549293518,2167944946,837.044381,2167.944946,0500000US09001
660,09,007,Middlesex,956493754,180679140,1137172894,439.062893,1137.172894,0500000US09007
661,09,011,New London,1722716728,276657755,1999374483,771.959260,1999.374483,0500000US09011
1238,09,005,Litchfield,2384116952,62334525,2446451477,944.575860,2446.451477,0500000US09005
1239,09,013,Tolland,1062807467,17549693,1080357160,417.126317,1080.357160,0500000US09013
1991,09,015,Windham,1328478475,21477921,1349956396,521.218686,1349.956396,0500000US09015
2003,09,003,Hartford,1903543897,40543777,1944087674,750.613002,1944.087674,0500000US09003


In [139]:
census_data_size.dtypes

COUNTY                                   object
FIPS                                     object
POPULATION_WHITE_NOT_LATINO_ALL         float64
POPULATION_WHITE_NOT_LATINO_MALE        float64
POPULATION_WHITE_NOT_LATINO_FEMALE      float64
POPULATION_HISPANIC_OR_LATINO_ALL       float64
POPULATION_HISPANIC_OR_LATINO_MALE      float64
POPULATION_HISPANIC_OR_LATINO_FEMALE    float64
AGE_MEDIAN_ALL_MALE                     float64
AGE_MEDIAN_ALL_FEMALE                   float64
POPULATION_TOTAL                        float64
INCOME_MEDIAN_INCOME_12_MONTH           float64
POPULATION_TOTAL_MALE                   float64
POPULATION_TOTAL_FEMALE                 float64
POPULATION_BLACK_ALL                    float64
POPULATION_BLACK_MALE                   float64
POPULATION_BLACK_FEMALE                 float64
GEO_STATE_FIPS                           object
GEO_COUNTY_FIPS                          object
YEAR                                    float64
POPULATION_DEGREE                       

In [141]:
census_data_size['POPULATION_DENSITY'] = census_data_size.POPULATION_TOTAL / census_data_size.area_sqmi
census_data_size['RATIO_MALE'] = census_data_size.POPULATION_TOTAL_MALE / census_data_size.POPULATION_TOTAL
census_data_size['RATIO_FEMALE'] = census_data_size.POPULATION_TOTAL_FEMALE / census_data_size.POPULATION_TOTAL
census_data_size['RATIO_WHITE'] = census_data_size.POPULATION_WHITE_NOT_LATINO_ALL / census_data_size.POPULATION_TOTAL
census_data_size['RATIO_HISPANIC'] = census_data_size.POPULATION_HISPANIC_OR_LATINO_ALL / census_data_size.POPULATION_TOTAL
census_data_size['POPULATION_BLACK_ALL'] = census_data_size.POPULATION_HISPANIC_OR_LATINO_ALL / census_data_size.POPULATION_TOTAL
census_data_size['POPULATION_DEGREE'] = census_data_size.POPULATION_HISPANIC_OR_LATINO_ALL / census_data_size.POPULATION_TOTAL

In [144]:
census_data_size.to_csv('C:/Users/appar/OneDrive/jup_nb/census_data.csv')